In [1]:
import pandas as pd
from glob import glob
from functools import reduce

In [2]:
### Categorical mappings for submission format
MAPS = {
    'ADNC': {0:"Not AD",1:"Low",2:"Intermediate",3:"High"},
    'Braak': {i:f"Braak {['0','I','II','III','IV','V','VI'][i]}" for i in range(7)},
    'CERAD': {0:"Absent",1:"Sparse",2:"Moderate",3:"Frequent"},
    'Thal': {i:f"Thal {i}" for i in range(6)}
}

In [3]:
### Process predictions to submission format: categorical to strings, continuous to float [0,100]
def format_evaluation(f):

    ## Extract target name from filename (format: {region}_{target}_train_predictions.csv)
    parts = f.split('/')[-1].replace('_test_predictions.csv', '').split('_')
    target = '_'.join(parts[1:])
    
    ## Load test predictions and ensure data type (string) in Donor ID
    df = pd.read_csv(f)[['Donor ID', 'predicted_value']]
    df['Donor ID'] = df['Donor ID'].astype(str) 
    
    ## Convert predictions: categorical to strings, continuous to clipped float [0,100]
    if target in MAPS:
        df['predicted_value'] = df['predicted_value'].round().astype(int).map(MAPS[target])
    else:
        df['predicted_value'] = df['predicted_value'].clip(0, 100).astype(float)
    
    ## Format column name for submission
    col_name = f"predicted {target.replace('percent_','').replace('_positive_area','').replace('percent ','').replace(' positive area','')}"
    
    return df.rename(columns={'predicted_value': col_name})

In [4]:
### Process all predictions per target and region
for region in ['MTG', 'A9']:
    
    targets = ['ADNC', 'Braak', 'CERAD', 'Thal', 
               'percent 6e10 positive area', 'percent AT8 positive area', 
               'percent GFAP positive area', 'percent NeuN positive area']
    
    files = [f'./output/{region}/{region}_{t}_test_predictions.csv' for t in targets]
    
    submission = reduce(lambda l,r: l.merge(r, on='Donor ID'), map(format_evaluation, files))
    
    submission.to_csv(f'./output/{region}/submission_{region}_test.csv', index=False)